## 0. Define a Specification

In the first phase of SDMT, we define a `Specification` that represents the requirements the completed model must meet in order to be acceptable for use in the system into which it will be integrated.

### Preliminaries

In [1]:
# Preliminaries for loading the package locally
import os
import sys

def package_root() -> str:
    """Resolve the path to the project root."""
    return os.path.abspath(os.path.join(os.getcwd(), "..", "src/"))

sys.path.append(package_root())

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [2]:
import mlte
import os

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(store_path, exist_ok=True)   # Ensure we are creating the folder if it is not there.

mlte.set_model("IrisClassifier", "0.0.1")
mlte.set_artifact_store_uri(f"local://{store_path}")

#### Build a `Specification`

In MLTE, we define requirements by constructing a specification (`Spec`).

In [3]:
from mlte.spec import Spec

from mlte.property.costs import (
    StorageCost,
    TrainingComputeCost,
    TrainingMemoryCost
)
from mlte.property.functionality import TaskEfficacy

spec = Spec(properties=[
    TaskEfficacy(),
    StorageCost(),
    #TrainingMemoryCost(),
    TrainingComputeCost()])
spec.save()

In [ ]:
from mlte.spec import Spec, Condition
from mlte.measurement import ExternalMeasurement
from mlte.measurement.result import Real, Image
from confusion_matrix import ConfusionMatrix
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.storage import LocalObjectSize

from mlte.property.costs import (
    StorageCost,
    TrainingComputeCost,
    TrainingMemoryCost
)
from mlte.property.functionality import TaskEfficacy

spec = Spec(property_conditions=
    {
        TaskEfficacy(): [Condition(ExternalMeasurement("accuracy", Real).metadata, "greater_or_equal_to", 0.9),
                         Condition(ExternalMeasurement("confusion matrix", ConfusionMatrix).metadata, "misclassification_count_less_than", 2),
                         Condition(ExternalMeasurement("class distribution", Image).metadata, "ignore", "Inspect the image.")],
        StorageCost(): [Condition(LocalObjectSize("model size").metadata, "less_than", 3000)],
        #TrainingMemoryCost(),
        TrainingComputeCost(): [Condition(LocalProcessCPUUtilization("training cpu").metadata, "max_utilization_less_than", 5.0)]
    }
)
spec.save()